<a href="https://colab.research.google.com/github/gmcalixto/llm-military-dictatorship-analysis/blob/main/Inquiring_AI_Models_sensible_topics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Notebook - análise comparativa de respostas de modelos de LLM a perguntas sobre temas sensíveis

#Instalando dependências

In [ ]:
# Instalar as bibliotecas necessárias (se ainda não estiverem instaladas)
!pip install transformers sentence-transformers spacy scikit-learn seaborn matplotlib
!python -m spacy download pt_core_news_sm

#Conectando ao Google Drive e gerando diretórios

In [ ]:
#Importando Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
# Define o diretório base
base_directory = "/content/drive/MyDrive/Universidade/UNEB/Projetos/DataLab/Data_Libray/Biblioteca_projetos/LLM_USP/Entrevistando_AI/"
os.makedirs(base_directory, exist_ok=True)

base_directory_csv = os.path.join(base_directory, 'csv') #salvar os logs
os.makedirs(base_directory_csv, exist_ok=True)

base_directory_viz = os.path.join(base_directory, 'visualizacao') #salvar os csvs
os.makedirs(base_directory_viz, exist_ok=True)

abrindo base de dados

In [ ]:
import os
import pandas as pd

# Definição do diretório base (já definido anteriormente)
file_entrevista = os.path.join(base_directory, "entrevistando AI.csv")
file_alan = os.path.join(base_directory, "analise_texto_alan (1).csv")

# Carregar os arquivos como DataFrames
df_entrevista = pd.read_csv(file_entrevista)
df_alan = pd.read_csv(file_alan)

# Exibir as primeiras linhas para verificar o carregamento
print("df_entrevista:")
display(df_entrevista.head())

print("\ndf_alan:")
display(df_alan.head())


##Abordagem

Com base nessa planilha: https://docs.google.com/spreadsheets/d/1gaEFQ3_ZtcLqv8CVQMlm6Q2rEivubRSt/edit?gid=386360956#gid=386360956

Foram escolhidos textos que poderiam ser classificados com discurso (1) favorável ao regime militar, (2) desfavorável ao regime militar e (3) neutro ao regime militar.

Foram escolhidos 3 textos para análise (destacados em amarelo na planilha):
“Assassinato leva estudantes à greve nacional”, presente na linha 4 da planilha, como “desfavorável”
“A plenitude democrática”, presente na linha 15, como “favorável”
“A história do Ato Institucional n.º 5, presente na linha 19, como “neutro”

- Submeti os 3 textos ao GPT-4 (4o) (OpenAI), Sabiá-3 (MaritacaAI) e DeepSeek-R1 com o seguinte prompt: “o texto vindo de jornal brasileiro tem viés favorável ou crítico ao regime militar?”

Compilei os resultados na planilha “Resultados das respostas”, indicando os links dos chats gerados no ChatGPT e DeepSeek (menos o Sabiá-3 que não permite gerar link) e reproduzindo as respostas em todos: https://docs.google.com/spreadsheets/d/1XbzPzMsSaQCCCl4AwrpfHaLLe5gxaCE_R-dLjaskfBE/edit?gid=0#gid=0

Possíveis pontos de discussão a partir dos resultados:
de quais maneiras as respostas dos três modelos coincidem ou divergem da classificação humana? apresentam coerência interna ou respostas contraditórias? quais termos utilizados em cada modelo permitem inferir sutis ou explicitos viéses nas respostas? Há padrões linguísticos ou argumentos recorrentes que revelam um viés implícito?
como as diferenças nas bases e treinamento dos modelos parecem influenciar a percepção de viés nos textos?
nova formulação do prompt influencia as respostas dos modelos em qual medida? (não testamos)
quais termos aparecem com mais frequência em textos classificados como favoráveis, desfavoráveis e neutros? Os modelos são sensíveis a certas palavras ao determinar o viés?


In [ ]:
# Exemplo de uso no Colab ou em um script Python:

# 1) Importe ou cole este código em uma célula.
# 2) Carregue seu DataFrame (por exemplo, a partir de um CSV):
df = df_alan.copy()

# 3) Chame a função:
result_dict = analyze_model_responses(
    df=df,
    question_col="texto",
    model_cols=['ChatGPT', 'Sabiá-3', 'DeepSeek'],
    theme_col="posicao",              # Se não existir a coluna 'tema', passe None
    output_dir=base_directory # Diretório de saída
)

# 4) A função retorna um dicionário com DataFrames, caso queira manipular além do salvamento:
#df_sent = result_dict["df_sentiment"]


In [ ]:

# Salvar cada DataFrame como variável independente no ambiente Colab
df_sentiment = result_dict["df_sentiment"]
df_length_classification = result_dict["df_length_classification"]
df_similarity_by_question = result_dict["df_similarity_by_question"]
df_top_ngrams_by_question_model = result_dict["df_top_ngrams_by_question_model"]
df_overall_model_stats = result_dict["df_overall_model_stats"]
df_global_similarity = result_dict["df_global_similarity"]

# Exibir os DataFrames carregados
print("DataFrames disponíveis no ambiente:")
for name, df in result_dict.items():
    print(f"- {name} (shape: {df.shape})")


display(df_sentiment.head(5))
display(df_length_classification.head(5))
display(df_similarity_by_question.head(5))
display(df_top_ngrams_by_question_model.head(5))
display(df_overall_model_stats.head(5))
display(df_global_similarity.head(5))




### **Descrição do desenho do estudo e metodologia de análise**

O estudo busca explorar como diferentes modelos de linguagem posicionam-se sobre temas politicamente sensíveis, com foco na influência de viéses associados ao contexto geopolítico de treinamento e implementação desses modelos. Para isso, foram formuladas perguntas diretas e objetivas sobre escravidão, ditaduras e sistemas de governo no Brasil, China e Estados Unidos. Além disso, uma questão final investigou a posição dos modelos sobre liberdades individuais, de expressão e de manifestação política nesses países. As perguntas foram aplicadas nas interfaces web oficiais dos serviços, considerando versões gratuitas para DeepSeek e Sabiá 3, e a versão padrão do GPT-4o. A metodologia adotou engenharia de prompt para evidenciar possíveis restrições impostas por censura ou filtragem de conteúdo, particularmente no caso da DeepSeek, que demonstrou bloqueios ao tratar de temas como ditaduras na China.

As respostas foram analisadas com base em múltiplos critérios, incluindo o tamanho da resposta, vocabulário empregado, similaridade semântica e padrões lexicais. Para isso, foram utilizados modelos de embeddings (*SentenceTransformer*) para aferir proximidade entre respostas e identificar variações na formulação de conteúdos pelos modelos. Foram extraídas *n-grams* e estatísticas de vocabulário, permitindo comparar a riqueza lexical e possíveis diferenças na forma de construção das respostas. A análise de sentimento foi inicialmente considerada, mas posteriormente descartada devido a inconsistências nas classificações entre diferentes modelos (*RoBERTa* e *BERT-multilingual*). Os resultados foram sistematicamente armazenados em um dataframe, permitindo a comparação estruturada entre os modelos e facilitando a visualização de padrões e tendências.

#Visualização de dados dos vocabulários usados e calculo do C-TF-IDF dos termos

#Analise lexical por c-tf-idf para identificação das marcas lexicais de cada modelo (tokens e bigramas)

In [ ]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import files
from IPython.display import display

# 1. Upload CSV file
uploaded = files.upload()  # Faça o upload do arquivo CSV
file_name = list(uploaded.keys())[0]
df = pd.read_csv(file_name)

# 2. Filtrar os modelos desejados
models = ["ChatGPT", "Sabiá-3", "DeepSeek"]
df_filtered = df[df["modelo"].isin(models)].copy()

# 3. Solicitar ao usuário o nome da coluna contendo o texto/tokens a serem avaliados (default: "tokens")
column_name = input("Enter the column name containing the text/tokens to evaluate (default 'tokens'): ").strip()
if column_name == "":
    column_name = "tokens"
# Se o usuário inserir mais de um nome (separados por vírgula), usa apenas o primeiro.
if ',' in column_name:
    column_name = column_name.split(',')[0].strip()

# 4. Converter a coluna especificada de string para lista (caso esteja em formato de string)
df_filtered[column_name] = df_filtered[column_name].apply(ast.literal_eval)

# 5. Agrupar os tokens (ou bigrams) por modelo, tratando cada item da lista como uma unidade inteira
def aggregate_texts(series):
    # "series" contém listas de tokens ou bigrams em cada linha
    all_tokens = []
    for tokens in series:
        # tokens é uma lista; assumimos que cada item já é uma string completa (ex: 'jornal do brasil')
        all_tokens.extend(tokens)
    # Usa um delimitador pouco provável de ocorrer ("|||") para preservar os termos
    return "|||".join(all_tokens)

df_grouped = df_filtered.groupby("modelo")[column_name].apply(aggregate_texts).reset_index()

# 6. Solicitar ao usuário as palavras a serem excluídas (ex.: "regime, texto")
excluded_words_input = input("Enter words to exclude (comma separated, e.g., regime, texto). Press ENTER to skip: ")
excluded_words = [w.strip().lower() for w in excluded_words_input.split(",") if w.strip()]

# 7. Solicitar ao usuário o fator de escala para os valores do c-TF-IDF (default: 1)
scale_factor_input = input("Enter multiplication factor for scaling c-TF-IDF values (default 1): ").strip()
try:
    scale_factor = float(scale_factor_input) if scale_factor_input else 1.0
except:
    scale_factor = 1.0

# 8. Configurar o CountVectorizer para usar o delimitador "|||"
#    Ao definir token_pattern=None, o vectorizer usará o tokenizer fornecido.
if excluded_words:
    vectorizer = CountVectorizer(tokenizer=lambda s: s.split("|||"),
                                 token_pattern=None,
                                 stop_words=excluded_words)
else:
    vectorizer = CountVectorizer(tokenizer=lambda s: s.split("|||"),
                                 token_pattern=None)

# 9. Criar a matriz de contagem usando a coluna agregada
count_matrix = vectorizer.fit_transform(df_grouped[column_name])
terms = vectorizer.get_feature_names_out()

# 10. Calcular c-TF-IDF
# 10.1. c-TF: frequência normalizada dos termos por documento (modelo)
count_array = count_matrix.toarray()
row_sums = count_array.sum(axis=1, keepdims=True)
tf = count_array / (row_sums + 1e-9)  # Evita divisão por zero

# 10.2. c-IDF: log((N+1)/(df_term+1)) + 1, onde df_term é o número de modelos que contêm o termo
df_term = np.count_nonzero(count_array, axis=0)
N = len(df_grouped)
cidf = np.log((N + 1) / (df_term + 1)) + 1

# 10.3. c-TF-IDF: multiplica c-TF por c-IDF e aplica o fator de escala
ctfidf = (tf * cidf) * scale_factor

# 11. Função para plotar os 10 principais termos de um modelo sem bordas e com rótulos numéricos
def plot_ctfidf(model_index, model_name):
    row_values = ctfidf[model_index]
    top_indices = row_values.argsort()[::-1][:10]
    top_terms = [terms[i] for i in top_indices]
    top_scores = [row_values[i] for i in top_indices]

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(top_terms, top_scores, color="steelblue", alpha=0.8)
    ax.set_xlabel("c-TF-IDF Score")
    ax.set_title(f"Top 10 Characteristic Terms for {model_name}")
    ax.invert_yaxis()  # Maior valor no topo

    # Remover todas as bordas (spines)
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Adicionar rótulos numéricos em cada barra
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 0.001, bar.get_y() + bar.get_height()/2,
                f'{width:.3f}', va='center', fontsize=10)

    plt.tight_layout()
    plt.show()

# 12. Plotar os gráficos para cada modelo
for i, model in enumerate(df_grouped["modelo"]):
    plot_ctfidf(i, model)

# 13. Criar um DataFrame comparando os 3 principais termos para cada modelo com seus scores e a variância
data = []
for i, model in enumerate(df_grouped["modelo"]):
    row_values = ctfidf[i]
    top_indices = row_values.argsort()[::-1][:3]
    top_terms = [terms[idx] for idx in top_indices]
    top_scores = [row_values[idx] for idx in top_indices]
    variance = np.var(top_scores)
    data.append({
        "Model": model,
        "Top Term 1": top_terms[0],
        "Score 1": top_scores[0],
        "Top Term 2": top_terms[1],
        "Score 2": top_scores[1],
        "Top Term 3": top_terms[2],
        "Score 3": top_scores[2],
        "Variance": variance
    })

df_top3 = pd.DataFrame(data)
print("Top 3 Characteristic Terms by Model with c-TF-IDF Score Variance:")
display(df_top3)
